In [24]:
import xml.etree.ElementTree as ET
from xml.etree.ElementTree import Element, SubElement, Comment
from lxml import etree
from copy import copy
import os
from xml.dom import minidom
import pandas as pd
import numpy as np
import sys
import datetime

# Get Unique Stop IDs

In [20]:
def get_unique_stops(trip_file):  
    '''
    This function takes a relative or explicit path to your trip file and returns only the unique stop location 
    names to be used to geneerate your additional file.
    '''
    unique_stops = set()
    tree = etree.parse(trip_file)
    root = tree.getroot()
    for trip in root.findall('trip'):
        stop = trip.find('stop')
        if stop != None:
            unique_stops.add(stop.attrib['busStop'])
    return unique_stops




# Build Additonal File With Unique IDs

In [30]:
def build_additional_file(unique_stops):
    '''
    Takes in unique stop IDs and builds out everything you'll need for your additional file and guarantees 
    all stop IDs are generated. Adds both vehicle types and their respective parameters along with vehicle stops.
    '''
    additional = Element('additional')
    additional.set('xmlns:xsi','http://www.w3.org/2001/XMLSchema-instance')
    additional.set('xsi:noNamespaceSchemaLocation', 'http://sumo.dlr.de/xsd/routes_file.xsd')
    vTypes = ['truck','trailer','bus','passenger']
    sigma = .13
    for veh in vTypes:
        
        vType = Element('vType')
        vType.set('id', veh)
        vType.set('vClass', veh)
        vType.set('sigma', str(sigma))
        if veh == 'passenger':
            vType.set('accel',str(10))
            vType.set('decel',str(10))
            vType.set('length',str(7))
        additional.append(vType)
        
    for stop in unique_stops:
        lane = stop + "_0"
        busStop = Element('busStop')
        busStop.set('endPos', "-1")
        busStop.set('friendlyPos', "1")
        busStop.set('id', stop)
        busStop.set('lane', lane)
        busStop.set('lines', str(1))
        busStop.set('startPos', str(0))
        additional.append(busStop)
    Date = str(datetime.datetime.today()).split()[0]
    file_name = Date + ".additional.xml"
    folder = "../Example_Files/SUMO_Input_Files"
        
    with open(os.path.join(folder,file_name), 'wb') as f:
        f.write(minidom.parseString(ET.tostring(additional)).toprettyxml(encoding="utf-8"))
    print("your file named: ", file_name, " is located in: ",folder)

# Main Function

In [31]:
def main(trip_file):
    unique_stops = get_unique_stops(trip_file)
    build_additional_file(unique_stops)

# Run Code

In [32]:
# this is what geenerate in your master function
path_to_trip_file = "../Example_Files/TempInputTrips/2018-4-23.Scaled34%.trips.xml" 
main(path_to_trip_file)

your file named:  2020-04-09.additional.xml  is located in:  ../Example_Files/SUMO_Input_Files
